In [1]:
using BenchmarkTools
using StellarChem
using StellarConstants
using StellarEOS
using StellarOpacity
using StellarEvolution

In [2]:
isotope_list = StellarChem.get_isotope_list()

Dict{Symbol, Isotope} with 3352 entries:
  :Ne26  => Isotope(10, 26, "Ne", 26.0005)
  :C22   => Isotope(6, 22, "C", 22.0575)
  :Pr143 => Isotope(59, 143, "Pr", 142.911)
  :Re163 => Isotope(75, 163, "Re", 162.972)
  :Cu52  => Isotope(29, 52, "Cu", 51.9967)
  :Co50  => Isotope(27, 50, "Co", 49.9809)
  :Sr74  => Isotope(38, 74, "Sr", 73.9562)
  :Pt188 => Isotope(78, 188, "Pt", 187.959)
  :No258 => Isotope(102, 258, "No", 258.098)
  :Pb210 => Isotope(82, 210, "Pb", 209.984)
  :Dy148 => Isotope(66, 148, "Dy", 147.927)
  :Hg200 => Isotope(80, 200, "Hg", 199.968)
  :Ir182 => Isotope(77, 182, "Ir", 181.958)
  :Md246 => Isotope(101, 246, "Md", 246.082)
  :Lr251 => Isotope(103, 251, "Lr", 251.094)
  :Cu75  => Isotope(29, 75, "Cu", 74.9415)
  :Ce144 => Isotope(58, 144, "Ce", 143.914)
  :Xe125 => Isotope(54, 125, "Xe", 124.906)
  :Ni66  => Isotope(28, 66, "Ni", 65.9291)
  ⋮      => ⋮

In [3]:
function equationHSE(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                            eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                            κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    if k==sm.nz
        lnP₀ = var00[sm.vari[:lnP]]
        r₀ = exp(var00[sm.vari[:lnr]])
        g₀ = CGRAV*sm.mstar/r₀^2
        return lnP₀ -log(2g₀/(3κ00)) # Eddington gray, ignoring radiation pressure term
    end
    lnP₊ = varp1[sm.vari[:lnP]]
    lnP₀ = var00[sm.vari[:lnP]]
    lnPface = (sm.dm[k]*lnP₀ + sm.dm[k+1]*lnP₊)/(sm.dm[k]+sm.dm[k+1])
    r₀ = exp(var00[sm.vari[:lnr]])
    dm = (sm.m[k+1]-sm.m[k])
    
    return (exp(lnPface)*(lnP₊ - lnP₀)/dm + CGRAV*sm.m[k]/(4π*r₀^4))/(CGRAV*sm.m[k]/(4π*r₀^4))
end

function equationT(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                          eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                          κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    if k==sm.nz
        lnT₀ = var00[sm.vari[:lnT]]
        L₀ = var00[sm.vari[:lum]]*LSUN
        r₀ = exp(var00[sm.vari[:lnr]])
        return lnT₀ - log(L₀/(BOLTZ_SIGMA*4π*r₀^2))/4 # Eddington gray, ignoring radiation pressure term
    end
    κface = exp((sm.dm[k]*log(κ00) + sm.dm[k+1]*log(κp1))/(sm.dm[k]+sm.dm[k+1]))
    L₀ = var00[sm.vari[:lum]]*LSUN
    r₀ = exp(var00[sm.vari[:lnr]])
    Pface = exp((sm.dm[k]*var00[sm.vari[:lnP]] + sm.dm[k+1]*varp1[sm.vari[:lnP]])/(sm.dm[k]+sm.dm[k+1]))
    lnT₊ = varp1[sm.vari[:lnT]]
    lnT₀ = var00[sm.vari[:lnT]]
    Tface = exp((sm.dm[k]*lnT₀ + sm.dm[k+1]*lnT₊)/(sm.dm[k]+sm.dm[k+1]))

    ∇ᵣ = 3κface*L₀*Pface/(16π*CRAD*CLIGHT*CGRAV*sm.m[k]*Tface^4)
    ∇ₐ = (sm.dm[k]*eos00[7] + sm.dm[k+1]*eosp1[7])/(sm.dm[k]+sm.dm[k+1])

    if (∇ᵣ < ∇ₐ)
        return (Tface*(lnT₊ - lnT₀)/sm.dm[k] + CGRAV*sm.m[k]*Tface/(4π*r₀^4*Pface)*∇ᵣ)/(CGRAV*sm.m[k]*Tface/(4π*r₀^4*Pface)) # only radiative transport
    else
        return (Tface*(lnT₊ - lnT₀)/sm.dm[k] + CGRAV*sm.m[k]*Tface/(4π*r₀^4*Pface)*∇ₐ)/(CGRAV*sm.m[k]*Tface/(4π*r₀^4*Pface)) # only radiative transport
    end
end

function equationLuminosity(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                          eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                          κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    L₋::TT = 0 # central luminosity is zero at first cell
    if k>1
        L₋ = varm1[sm.vari[:lum]]*LSUN # change it if not at first cell
    end
    L₀ = var00[sm.vari[:lum]]*LSUN
    ρ₀ = eos00[1]
    cₚ = eos00[5]
    δ = eos00[6]
    dTdt = (exp(var00[sm.vari[:lnT]]) - exp(sm.ssi.lnT[k]))/sm.ssi.dt
    dPdt = (exp(var00[sm.vari[:lnP]]) - exp(sm.ssi.lnP[k]))/sm.ssi.dt

    ϵnuc = 0.1*var00[sm.vari[:H1]]^2*ρ₀*(exp(var00[sm.vari[:lnT]])/1e6)^4 + 0.1*var00[sm.vari[:H1]]*ρ₀*(exp(var00[sm.vari[:lnT]])/1e7)^18

    return ((L₀-L₋)/sm.dm[k]-ϵnuc+cₚ*dTdt -(δ/ρ₀)*dPdt) # no nuclear reactions or neutrinos
end

function equationContinuity(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                                   eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                                   κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    ρ₀ = eos00[1]
    r₀ = exp(var00[sm.vari[:lnr]])
    r₋::TT = 0 # central radius is zero at first cell
    if k>1
        r₋ = exp(varm1[sm.vari[:lnr]]) # change it if not at first cell
    end
    
    dm = sm.m[k] # this is only valid for k=1
    if k>1
        dm = dm-sm.m[k-1]
    end

    #expected_r₀ = r₋ + dm/(4π*r₋^2*ρ)
    expected_dr³_dm = 3/(4π*ρ₀)
    actual_dr³_dm = (r₀^3-r₋^3)/dm
    
    return (expected_dr³_dm - actual_dr³_dm)*ρ₀
end

#  To test performance, include 8 isotopes similar to basic.net in MESA.
#  of course we are keeping these fixed now, but it lets us test their impact on the
#  computation of the jacobian

function equationH1(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                                    eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                                    κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    ρ₀ = eos00[1]
    ϵnuc = 0.1*var00[sm.vari[:H1]]^2*ρ₀*(exp(var00[sm.vari[:lnT]])/1e6)^4 + 0.1*var00[sm.vari[:H1]]*ρ₀*(exp(var00[sm.vari[:lnT]])/1e7)^18
    rate_per_unit_mass = 4*ϵnuc/((4*isotope_list[:H1].mass-isotope_list[:He4].mass)*AMU*CLIGHT^2)

    Xi = sm.ssi.ind_vars[(k-1)*sm.nvars+sm.vari[:H1]]

    return (var00[sm.vari[:H1]]-Xi)/sm.ssi.dt + isotope_list[:H1].mass*AMU*rate_per_unit_mass
end

function equationHe4(sm, k, varm1::Vector{<:TT}, var00::Vector{<:TT}, varp1::Vector{<:TT},
                                    eosm1::Vector{<:TT}, eos00::Vector{<:TT}, eosp1::Vector{<:TT},
                                    κm1::TT, κ00::TT, κp1::TT)::TT where{TT<:Real}
    return  var00[sm.vari[:He4]] + var00[sm.vari[:H1]] - 1.0
end

equationHe4 (generic function with 1 method)

In [4]:
nvars = 6
nspecies = 2
varnames = [:lnP,:lnT,:lnr,:lum,:H1, :He4]
structure_equations=[equationHSE, equationT,
                        equationContinuity, equationLuminosity,
                        equationH1, equationHe4]
nz = 1000
eos = StellarEOS.IdealEOS(false)
opacity = StellarOpacity.SimpleElectronScatteringOpacity()
sm = StellarModel(varnames, structure_equations, nvars, nspecies, nz, eos, opacity);

In [5]:
StellarEvolution.n1_polytrope_initial_condition(sm, MSUN, 100*RSUN; initial_dt=0.01*SECYEAR)

StellarEvolution.set_end_step_info(sm)
StellarEvolution.cycle_step_info(sm)
StellarEvolution.set_start_step_info(sm)

StellarEvolution.eval_jacobian!(sm)
StellarEvolution.eval_eqs!(sm)

In [6]:
using LinearSolve
@benchmark begin
    $sm.linear_solver.A = $sm.jac
    $sm.linear_solver.b = $sm.eqs
    corr =solve($sm.linear_solver)
end

BenchmarkTools.Trial: 824 samples with 1 evaluation.
 Range (min … max):  5.125 ms …   8.483 ms  ┊ GC (min … max): 0.00% … 16.40%
 Time  (median):     5.779 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.053 ms ± 692.664 μs  ┊ GC (mean ± σ):  4.05% ±  6.86%

       ▂▁▇▇▇█▆▇▂▃▂▂                                            
  ▂▂▃▄▆██████████████▆▄▅▂▃▄▄▄▂▂▂▁▃▂▂▂▄▄▄▄▅▆▅▄▆▅▄▄▃▃▃▃▂▂▁▃▂▂▃▃ ▄
  5.13 ms         Histogram: frequency by time        8.01 ms <

 Memory estimate: 8.73 MiB, allocs estimate: 41.

In [7]:
@benchmark StellarEvolution.eval_jacobian_row!(sm,1)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.200 μs … 653.914 μs  ┊ GC (min … max): 0.00% … 95.67%
 Time  (median):      8.874 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.639 μs ±  24.726 μs  ┊ GC (mean ± σ):  9.25% ±  3.93%

  ▂▆▇█▇▆▃▁ ▁▁▂▂▂▁        ▁▁                                    ▂
  ████████████████▇▇▇▇▇███████▇█▇▇▇████████▇▇▇▇▆▆▆▄▆▆▆▇▆▆▆▆▇▆▇ █
  8.2 μs        Histogram: log(frequency) by time      18.4 μs <

 Memory estimate: 25.28 KiB, allocs estimate: 211.

In [8]:
@benchmark StellarEvolution.eval_jacobian!(sm)

BenchmarkTools.Trial: 409 samples with 1 evaluation.
 Range (min … max):   4.264 ms … 57.536 ms  ┊ GC (min … max):  0.00% … 87.78%
 Time  (median):      7.531 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   12.238 ms ± 13.785 ms  ┊ GC (mean ± σ):  42.90% ± 28.79%

  ▃▂▃█▁                                            ▁           
  █████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▄▅▆██▆▄▁▁▁▄▁▁▅ ▆
  4.26 ms      Histogram: log(frequency) by time      56.8 ms <

 Memory estimate: 41.93 MiB, allocs estimate: 456470.

In [9]:
StellarEvolution.set_options!(sm.opt, "example_options.toml")
rm(sm.opt.io.hdf5_history_filename, force=true)
rm(sm.opt.io.hdf5_profile_filename, force=true)
StellarEvolution.n1_polytrope_initial_condition(sm, MSUN, 100*RSUN, initial_dt=1000*SECYEAR)

StellarEvolution.do_evolution_loop(sm)

(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (50, 3.0, 4392.773678888243, 1.8479847518548578e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (100, 3.0, 4935.13025378168, 2.354658929466138e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (150, 3.0, 10104.68689979219, 2.3077459357511185e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (200, 3.0000000000000004, 12245.175949058055, 2.277531470473329e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (250, 2.9999999999999996, 11136.940516364819, 2.249268656630319e7)


...

(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (800, 3.0, 3.518432362278031e6, 6.3630602208055675e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (850, 3.0, 2.7568925239477404e6, 6.378505015328043e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (900, 3.0, 1.785945247759834e6, 6.394438781217843e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (950, 3.0, 3.1929218974736664e6, 6.409527630951707e7)


(i, maximum(corr), real_max_corr, maximum(sm.eqs)) = (1000, 3.0, 3.0379924689693414e6, 6.424382628001423e7)
Failed to converge, retry


In [10]:
# Compare ρ-T profile with different polytropes

profile_names = StellarEvolution.get_profile_names_from_hdf5("profiles.hdf5")

using CairoMakie, LaTeXStrings

f = Figure()
ax = Axis(f[1,1], xlabel=L"\log_{10}(\mathrm{\rho/[g\;cm^{-3}]})", ylabel=L"\log_{10}(\mathrm{dyne]})")

pname = Observable(profile_names[1])

profile = @lift(StellarEvolution.get_profile_from_hdf5("profiles.hdf5", $pname))
log10_ρ = @lift($profile[!,"log10_ρ"])
log10_P = @lift($profile[!,"log10_P"])

profile_line = lines!(ax, log10_ρ, log10_P, label="real profile")
#compare profile against some polytropes
xvals = LinRange(-13,4,100)
lines!(ax,xvals,(1+1/1) .* xvals .+ 20, label="n=1")
lines!(ax,xvals,(1+1/(1.5)) .* xvals .+ 15, label="n=1.5")
lines!(ax,xvals,(1+1/3) .* xvals .+ 15, label="n=3")
axislegend(ax, position=:rb)

record(f, "rho_P_evolution.mp4", profile_names[1:end];
        framerate = 60) do profile_name
    pname[] = profile_name
end

f

ErrorException: unable to determine if profiles.hdf5 is accessible in the HDF5 format (file may not exist)

In [11]:
# Check evolution of abundance

profile_names = StellarEvolution.get_profile_names_from_hdf5("profiles.hdf5")

using CairoMakie

f = Figure()
ax = Axis(f[1,1], xlabel=L"\mathrm{Mass}\;[M_\odot]", ylabel=L"X")

pname = Observable(profile_names[1])

profile = @lift(StellarEvolution.get_profile_from_hdf5("profiles.hdf5", $pname))
mass = @lift($profile[!,"mass"])
X = @lift($profile[!,"X"])

profile_line = lines!(ax, mass, X, label="real profile")

record(f, "X_evolution.mp4", profile_names[1:end];
        framerate = 60) do profile_name
    pname[] = profile_name
end

f

ErrorException: unable to determine if profiles.hdf5 is accessible in the HDF5 format (file may not exist)

In [12]:
f = Figure()
ax = Axis(f[1,1], xlabel=L"\log_{10}(T_\mathrm{eff}/[K])", ylabel=L"\log_{10}(L/L_\odot)")
xvals = collect(LinRange(0,1,1000))
history = StellarEvolution.get_history_dataframe_from_hdf5("history.hdf5")
lines!(ax, log10.(history[!,"T_surf"]), log10.(history[!,"L_surf"]))
f

UndefVarError: UndefVarError: `Figure` not defined